In [ ]:
import torch
from torch.cuda import random
import torch.nn as nn
import torch.nn.functional as F
import os
import torch.optim as optim
from os import listdir
from os.path import isfile, join
from tracin.tracin_batched import save_tracin_checkpoint, load_tracin_checkpoint,  approximate_tracin_batched
import pandas as pd
from LSTM_clean.model import LSTM
import numpy as np
import re
from statistics import mean
import scipy.stats as stats
import pandas as pd
from sklearn.utils import shuffle
from copy import deepcopy

In [ ]:
OUTPUT_SIZE = 1743

In [ ]:
curr_dir = os.getcwd()
path = curr_dir + "/checkpoints_subset/"
checkpoints = []
with os.scandir(path) as listOfEntries:
    for entry in listOfEntries:
        # print all entries that are files
        if entry.is_file():
            checkpoints.append(os.path.join(path,entry.name))
last_checkpoint_epoch = max([re.sub('[^0-9]','', a)[2:] for a in checkpoints])
last_checkpoint = sorted(checkpoints)[-1][:-5] + str(last_checkpoint_epoch) + ".pt"

In [ ]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]='6'

cpu_device = torch.device("cpu")
print("CPU Device is ", cpu_device)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device is ", device)

In [ ]:
train = np.load(os.path.join(os.getcwd(), "data/twitch_sequence/train.data"), allow_pickle=True)

In [ ]:
model = LSTM(
        input_size=128,
        output_size=OUTPUT_SIZE,
        hidden_dim=64,
        n_layers=1,
        device=cpu_device,
    )

In [ ]:
train_num = len(train)
train_labels = []

for i in range(train_num):
    train_labels.append(train[i][1])
train = [train[i][0] for i in range(train_num)]